# Script for Visualising the Correspondence Network of CdS Over the Decades

This script can be used to visualise network data from the CdS correspondence. The visualisations are interactive and allow the user to comprehend the development over the decades.

It is essential to install the following packages for using this script (this can be done using the IDE or ``pip install <package>``:
* [pandas](https://pandas.pydata.org/docs/)
* [networkx](https://networkx.org)
* [hvplot](https://hvplot.holoviz.org)
* [holoviews](https://holoviews.org)

In [40]:
import pandas as pd
import networkx as nx
import hvplot.networkx as hvnx
import holoviews as hv
import hvplot.pandas # noqa
import panel as pn

pn.extension(sizing_mode='stretch_width')

### Reading in data from CSV

In [3]:
df = pd.read_csv('../data/network_data.csv')
df = df.rename(columns={"counts": "weights"})

### The data needs to be divided into subsets to enable a filtering with a slider set on top of the visualisation

In [38]:
# Create a mask of the dataframe with a group for each decade. Result is stored in a dictionary.
grouped_mask = df.groupby('decade')
grouped_mask = grouped_mask.groups

dict_keys([1790.0, 1800.0, 1810.0, 1820.0, 1830.0, 1840.0])


In [54]:
dfi = df.interactive(sizing_mode='stretch_width')
decade_slider = pn.widgets.FloatSlider(
    name='Decades',
    start=min(grouped_mask.keys()),
    end=max(grouped_mask.keys()),
)



dfi[dfi['decade'] > decade_slider].head(10)

In [ ]:
# Create graph from the dataframe that includes information about nodes and their connecting edges.

G = nx.from_pandas_edgelist(
    df[:50],
    "Verfasser",
    "Empfänger",
    edge_attr='weights',
    create_using=nx.MultiGraph(),
)

elarge = [(u, v) for (u, v, attr) in G.edges(data=True) if attr['weights'] > 30]
emedium =  [(u, v) for (u, v, attr) in G.edges(data=True) if (10 < attr['weights'] <= 30)]
esmall = [(u, v) for (u, v, attr) in G.edges(data=True) if attr['weights'] <= 10]

pos = nx.spring_layout(G)
nodes = hvnx.draw_networkx_nodes(
    G,
    pos,
    node_color='blue'
)
edges_large = hvnx.draw_networkx_edges(
    G,
    pos,
    arrowstyle='->',
    edgelist=elarge,
    edge_width=10,
    edge_cmap='Blues',
    colorbar=True,
)

edges_medium = hvnx.draw_networkx_edges(
    G,
    pos,
    arrowstyle='->',
    edgelist=emedium,
    edge_width=6,
    edge_cmap='Blues',
    colorbar=True,
)


edges_small = hvnx.draw_networkx_edges(
    G,
    pos,
    arrowstyle='->',
    edgelist=emedium,
    edge_width=3,
    edge_cmap='Blues',
    colorbar=True,
)

labels = hvnx.draw_networkx_labels(
    G,
    pos,
    font_size=20,
    font_family='sans-serif'
)


hvnx.show(edges_large * edges_medium * edges_small * nodes * labels)